In [37]:
import cv2
import face_recognition
import pymysql
import datetime

In [38]:
#Loading data from database
def load_from_db():
    known_faces = []
    known_names = []

    #connecting to Database : Image_Database
    mydb = pymysql.connect(host='localhost',user='root',password='',database='Image_Database')
    mycursor = mydb.cursor()

    #Fetch data from the Image_path_Table
    sql = "SELECT * FROM Image_path_Table"
    mycursor.execute(sql)
    results = mycursor.fetchall()

    for row in results:
        name = row[0]
        image_path = row[1]

        # # Checking name and img-path
        # print(f"Name: {name},Image Path: {image_path}")

        # Load image and compute face encoding
        image = face_recognition.load_image_file(image_path)
        face_encoding = face_recognition.face_encodings(image)[0]

        known_names.append(name)
        known_faces.append(face_encoding)

    return known_faces,known_names

# load_from_db()        


In [39]:
# # writing into database

# def write_to_db(name,current_time):
#     if name != "Unknown":

#         # connect to Image_Database
#         mydb = pymysql.connect(host='localhost',user='root',password='',database='Image_Database')
#         mycursor = mydb.cursor()

#         #Create table for attendence
#         sql = "CREATE TABLE IF NOT EXISTS Attendance_Table (Name varchar(30),Time varchar(10))"
#         mycursor.execute(sql)
#         mydb.commit()

#         #Inserting data into table
#         sql = "INSERT INTO Attendance_Table (Name, Time) VALUES (%s, %s)"
#         mycursor.execute(sql, (name, current_time))
#         mydb.commit()

# # write_to_db("John Doe", "14:30:00")

In [40]:
# writing into database not duplicate

def write_to_db(name, current_time):
    if name != "Unknown":
        # Connect to Image_Database
        mydb = pymysql.connect(host='localhost', user='root', password='', database='Image_Database')
        mycursor = mydb.cursor()

        # Create table for attendance
        sql = "CREATE TABLE IF NOT EXISTS Attendance_Table (Name varchar(30),Time varchar(10), Date DATE)"
        mycursor.execute(sql)
        mydb.commit()

        # taking current date
        current_date = datetime.datetime.now().strftime('%Y-%m-%d')

        # Check if the data already exists
        check_sql = "SELECT * FROM Attendance_Table WHERE Name = %s AND Date = %s"
        mycursor.execute(check_sql, (name, current_date))
        existing_data = mycursor.fetchone()

        # If data doesn't exist, insert it
        if not existing_data:
            insert_sql = "INSERT IGNORE INTO Attendance_Table (Name, Time,Date) VALUES (%s, %s, %s)"
            mycursor.execute(insert_sql, (name, current_time, current_date))
        
        mydb.commit()

# write_to_db("John Doe", "14:30:00")


In [41]:
known_face_encodings, known_face_names = load_from_db()

In [42]:
video_capture = cv2.VideoCapture(0)

recognized_set = set()

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Find all face locations and encodings in the current frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    recognized_names = []

    for face_encoding in face_encodings:
        # Compare face encodings with known faces
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        if True in matches:
            matched_index = matches.index(True)
            name = known_face_names[matched_index]

        recognized_names.append(name)

    for name in recognized_names:
        if name not in recognized_set:
            current_time = datetime.datetime.now().strftime('%H:%M:%S')
            write_to_db(name, current_time)
            recognized_set.add(name)

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, recognized_names):
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 1)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
video_capture.release()
cv2.destroyAllWindows()